## Deliverable 2. Create a Customer Travel Destinations Map.

In [31]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Samoded,RU,63.6083,40.5111,15.73,95,85,6.38,overcast clouds
1,1,Albany,US,42.6001,-73.9662,30.61,76,87,17.22,light snow
2,2,Georgetown,MY,5.4112,100.3354,87.75,70,40,4.61,scattered clouds
3,3,Champerico,GT,14.3000,-91.9167,76.93,81,80,4.65,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,55.27,53,100,6.53,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Georgetown,MY,5.4112,100.3354,87.75,70,40,4.61,scattered clouds
3,3,Champerico,GT,14.3000,-91.9167,76.93,81,80,4.65,broken clouds
5,5,Tecoanapa,MX,16.5167,-98.7500,75.83,85,27,4.38,scattered clouds
9,9,Bambous Virieux,MU,-20.3428,57.7575,82.65,57,20,11.50,few clouds
10,10,Sinnamary,GF,5.3833,-52.9500,78.64,87,99,9.75,overcast clouds
12,12,Souillac,MU,-20.5167,57.5167,82.78,57,20,11.50,few clouds
19,19,Rikitea,PF,-23.1203,-134.9692,76.17,73,100,14.94,overcast clouds
23,23,Ambulu,ID,-8.3450,113.6058,83.14,76,99,5.35,light rain
26,26,Eyl,SO,7.9803,49.8164,84.06,63,62,12.84,broken clouds
39,39,Kapaa,US,22.0752,-159.3190,76.98,73,40,9.22,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 2 to 419
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              112 non-null    int64  
 1   City                 112 non-null    object 
 2   Country              112 non-null    object 
 3   Lat                  112 non-null    float64
 4   Lng                  112 non-null    float64
 5   Max Temp             112 non-null    float64
 6   Humidity             112 non-null    int64  
 7   Cloudiness           112 non-null    int64  
 8   Wind Speed           112 non-null    float64
 9   Current Description  112 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 9.6+ KB


In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.eq('').sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Georgetown,MY,87.75,scattered clouds,5.4112,100.3354,
3,Champerico,GT,76.93,broken clouds,14.3000,-91.9167,
5,Tecoanapa,MX,75.83,scattered clouds,16.5167,-98.7500,
9,Bambous Virieux,MU,82.65,few clouds,-20.3428,57.7575,
10,Sinnamary,GF,78.64,overcast clouds,5.3833,-52.9500,
12,Souillac,MU,82.78,few clouds,-20.5167,57.5167,
19,Rikitea,PF,76.17,overcast clouds,-23.1203,-134.9692,
23,Ambulu,ID,83.14,light rain,-8.3450,113.6058,
26,Eyl,SO,84.06,broken clouds,7.9803,49.8164,
39,Kapaa,US,76.98,scattered clouds,22.0752,-159.3190,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# Determine if there are any empty rows.
hotel_df.eq('').sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [51]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [52]:
clean_hotel_df.eq('').sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [53]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Georgetown,MY,87.75,scattered clouds,5.4112,100.3354,Cititel Penang
3,Champerico,GT,76.93,broken clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
5,Tecoanapa,MX,75.83,scattered clouds,16.5167,-98.7500,Hotel Manglares
9,Bambous Virieux,MU,82.65,few clouds,-20.3428,57.7575,Casa Tia Villa
10,Sinnamary,GF,78.64,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
12,Souillac,MU,82.78,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
19,Rikitea,PF,76.17,overcast clouds,-23.1203,-134.9692,People ThankYou
23,Ambulu,ID,83.14,light rain,-8.3450,113.6058,Hotel Ambulu Jember
26,Eyl,SO,84.06,broken clouds,7.9803,49.8164,Amiin Hotel
39,Kapaa,US,76.98,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [54]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))